In [1]:
import os
import tqdm
import torch
import wandb
import random
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
from transformers import T5Tokenizer, T5ForConditionalGeneration, AdamW
from sklearn.metrics import accuracy_score, precision_score, recall_score

C:\Users\Sid\AppData\Local\Temp\ipykernel_30028\401704351.py:7: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

In [3]:
wandb.init(project="T5 Model FineTuning Yo!")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: siddiqmoideen07. Use `wandb login --relogin` to force relogin


In [4]:
dataStuff = pd.read_csv("../augmentedDataStuff.csv")
dataStuff

,OCR Contents,Category
0,Invoice Number: INV0786Hotel Name: Spice Garde...,Food Category - Restaurant Bill/Receipt
1,Invoice No- DW/1112/851166\n\nInvoice Date : 2...,Travel Category - Flight Ticket
2,Hotel: Accor\r\nCity: Delhi\r\nNumber of Days:...,Miscellaneous Category - Miscellaneous Bills
3,FOOD INN FINE DINE\nProprietor\nPER =NNIA REST...,Food Category - Restaurant Bill/Receipt
4,= Confirmed Booking Id: NF20115276689422\n\nSU...,Travel Category - Flight Ticket
...,...,...
5161,Indiandit\n\nSERIAL SNA AUTONOBI\n\nr .A.B.Roa...,Fuel Category - Fuel Bill/Receipt
5162,Base Fare\nDistance\nTime\n\nSubtotal\n\nCHARG...,Travel Category - Cab Bill/Receipt
5163,"Dear sanjay kumar,\n\nCongratdatons! Thark you...",Travel Category - Train Ticket
5164,Invoice Number: INV0780Hotel Name: Taste of It...,Food Category - Restaurant Bill/Receipt


In [5]:
clsStuff = list(set(dataStuff["Category"]))
clsStuff

['Travel Category - Cab Bill/Receipt',
 'Travel Category - Train Ticket',
 'Travel Category - Bus Ticket',
 'Travel Category - Flight Ticket',
 'Food Category - Restaurant Bill/Receipt',
 'Electronics Category - Electronic Gadgets Bill/Receipt',
 'Room Stay Category - Stay Bill/Receipt',
 'Miscellaneous Category - Miscellaneous Bills',
 'Fuel Category - Fuel Bill/Receipt']

In [6]:
labelEncoder = LabelEncoder()
labelEncoder.fit(clsStuff)
clsDict = dict(zip(clsStuff, labelEncoder.transform(clsStuff)))
clsDict

{'Travel Category - Cab Bill/Receipt': 6,
 'Travel Category - Train Ticket': 8,
 'Travel Category - Bus Ticket': 5,
 'Travel Category - Flight Ticket': 7,
 'Food Category - Restaurant Bill/Receipt': 1,
 'Electronics Category - Electronic Gadgets Bill/Receipt': 0,
 'Room Stay Category - Stay Bill/Receipt': 4,
 'Miscellaneous Category - Miscellaneous Bills': 3,
 'Fuel Category - Fuel Bill/Receipt': 2}

In [7]:
numLabels = len(clsStuff)
batchSize = 16
learningRate = 2e-5
epochs = 250

In [8]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cuda')

In [9]:
tokenizer = T5Tokenizer.from_pretrained("google-t5/t5-small")
model = T5ForConditionalGeneration.from_pretrained("google-t5/t5-small").to(device)

tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [10]:
wandb.watch(model)
wandb.config["learning_rate"] = learningRate
wandb.config["batch_size"] = batchSize

In [11]:
trainData, valData = train_test_split(dataStuff, test_size=0.2, random_state=42)
trainData['OCR Contents']

4711    Hotel: Hilton Worldwide\r\nCity: Amritsar\r\nN...
4184    goibibo sao\n\nFlight Ticket - Kolkata to Agar...
1670    Category: Utilities\r\nItem: Internet Bill\r\n...
99      Invoice Number: INV0810Hotel Name: The Hungry ...
3592    Invoice Number: INV0953Hotel Name: Gourmet Pal...
                              ...                        
4426    Invoice Number: INV0558Hotel Name: The Hungry ...
466     ticket ric 0: r07s220168 Gcleartrip\n\nBangalo...
3092    Category: Shopping\r\nItem: Accessories\r\nAmo...
3772    " ‘\n\nSREP iy\n\nCHOKHI DHANI PALACE HOTEL\n\...
860     Category: Healthcare\r\nItem: Medicines\r\nAmo...
Name: OCR Contents, Length: 4132, dtype: object

In [12]:
def prepData(data):
  train_inputs, train_masks, train_labels = [], [], []
  for item in zip(data["OCR Contents"], data["Category"]):
    text_content, category_label = item
    summary_text = f"{text_content} ({category_label})"
    encoded_data = tokenizer(summary_text, padding="max_length", truncation=True, max_length=512)
    train_inputs.append(encoded_data["input_ids"])
    train_masks.append(encoded_data["attention_mask"])
    train_labels.append(category_label)
  return train_inputs, train_masks, train_labels


In [13]:
def labels2Encoding(labelStuff):
    encodedLabels = labelEncoder.transform(labelStuff)
    return encodedLabels

In [14]:
trainInputs, trainMasks, trainLabels = prepData(trainData)
valInputs, valMasks, valLabels = prepData(valData)

In [15]:
trainLabels = labels2Encoding(trainLabels)
valLabels = labels2Encoding(valLabels)
trainLabels

array([3, 5, 3, ..., 3, 1, 3])

In [16]:
trainInputs = torch.tensor(trainInputs, dtype=torch.uint8).to(device)
trainMasks = torch.tensor(trainMasks, dtype=torch.uint8).to(device)
trainLabels = torch.tensor(trainLabels, dtype=torch.uint8).to(device)
valInputs = torch.tensor(valInputs, dtype=torch.uint8).to(device)
valMasks = torch.tensor(valMasks, dtype=torch.uint8).to(device)
valLabels = torch.tensor(valLabels, dtype=torch.uint8).to(device)

In [17]:
trainLabels

tensor([3, 5, 3,  ..., 3, 1, 3], device='cuda:0', dtype=torch.uint8)

In [18]:
trainMasks

tensor([[1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 1, 1, 1],
        [1, 1, 1,  ..., 0, 0, 0],
        ...,
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0],
        [1, 1, 1,  ..., 0, 0, 0]], device='cuda:0', dtype=torch.uint8)

In [19]:
trainInputs

tensor([[234,  10, 243,  ...,   0,   0,   0],
        [ 25,  23, 184,  ...,  33, 157,   1],
        [ 51,  10, 156,  ...,   0,   0,   0],
        ...,
        [ 51,  10, 238,  ...,   0,   0,   0],
        [ 96, 202,   3,  ...,   0,   0,   0],
        [ 51,  10, 126,  ...,   0,   0,   0]], device='cuda:0',
       dtype=torch.uint8)

In [20]:
print(trainMasks.max(), trainInputs.min())

tensor(1, device='cuda:0', dtype=torch.uint8) tensor(0, device='cuda:0', dtype=torch.uint8)


In [21]:
trainDataset = TensorDataset(trainInputs, trainMasks, trainLabels)
trainDataloader = DataLoader(trainDataset, batch_size=batchSize, shuffle=True)

In [22]:
valDataset = TensorDataset(valInputs, valMasks, valLabels)
valDataloader = DataLoader(valDataset, batch_size=batchSize)

In [23]:
optimizer = AdamW(model.parameters(), lr=learningRate)
criterion = torch.nn.CrossEntropyLoss()

c:\Users\Sid\AppData\Local\Programs\Python\Python39\lib\site-packages\transformers\optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [41]:
def train(model, optimizer, criterion, trainDataloader):
  model.train()
  total_loss = 0
  correct = 0
  for batch_idx, (input_ids, attention_mask, labels) in enumerate(trainDataloader):
    optimizer.zero_grad()
    input_ids = input_ids.long().to(device)
    outputs = model(input_ids=input_ids, attention_mask=attention_mask)  # Only input_ids and attention_mask needed

    loss = criterion(outputs.logits, labels)
    loss.backward()
    optimizer.step()
    total_loss += loss.item()
    predictions = torch.argmax(outputs.logits, dim=1)
    correct += (predictions == labels).sum().item()
  return total_loss / len(trainDataloader), correct / len(trainDataset)



In [42]:
def validate(model, criterion, valDataloader):
  model.eval()
  total_loss = 0
  correct = 0
  all_preds, all_labels = [], []
  with torch.no_grad():
    for batch_idx, (input_ids, attention_mask, labels) in enumerate(valDataloader):
      input_ids = input_ids.long().to(device)
      outputs = model(input_ids=input_ids, attention_mask=attention_mask)
      loss = criterion(outputs.logits, labels)
      total_loss += loss.item()
      predictions = torch.argmax(outputs.logits, dim=1)
      correct += (predictions == labels).sum().item()
      all_preds.extend(predictions.cpu().numpy())
      all_labels.extend(labels.cpu().numpy())
  accuracy = accuracy_score(all_labels, all_preds)
  precision = precision_score(all_labels, all_preds, average="weighted")
  recall = recall_score(all_labels, all_preds, average="weighted")
  return total_loss / len(valDataloader), accuracy, precision, recall

In [43]:
if not os.path.isdir("tunedModels"):
   os.makedirs("tunedModels")

In [44]:
for epoch in tqdm.trange(2, colour = "red", desc = "Epoch(s)"):
  train_loss, train_acc = train(model, optimizer, criterion, trainDataloader)
  val_loss, val_acc, val_precision, val_recall = validate(model, criterion, valDataloader)
  dataDict = {
    "Epoch": epoch+1,
    "Train Loss": train_loss,
    "Train Accuracy": train_acc,
    "Val Loss": val_loss,
    "Val Accuracy": val_acc,
    "Val Precision": val_precision,
    "Val Recall": val_recall
    }
  print(dataDict)
  #wandb.log(dataDict)
  model.save_pretrained("tunedModels/trainedT5model - Epoch {}".format(epoch))

Epoch(s):   0%|          | 0/2 [00:00<?, ?it/s]


ValueError: You have to specify either decoder_input_ids or decoder_inputs_embeds